In [1]:
%pip install selenium webdriver-manager beautifulsoup4 pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 39.2 MB/s eta 0:00:00


In [2]:
# Install Chrome browser
!apt-get update
!apt install -y chromium-browser

# Set up options for headless Chrome
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--window-size=1920,1080')

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.2 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,823 kB]
Get:14 https://r2u.s

In [3]:
%pip install chromedriver-autoinstaller

import chromedriver_autoinstaller

# Check if the current version of chromedriver exists
# and if it doesn't exist, download it automatically,
# then add chromedriver to path
chromedriver_autoinstaller.install()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

url = "https://freespeechcollective.in/free-speech-tracker/"

# Setup Chrome Driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.get(url)
time.sleep(3)

# Keep clicking "Load More" until it disappears
while True:
    try:
        load_more = driver.find_element(By.CSS_SELECTOR, "a.et_pb_button")  # May need slight selector adjust
        driver.execute_script("arguments[0].click();", load_more)
        time.sleep(2)
    except:
        print("No more posts to load.")
        break

# Parse final HTML
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

posts = soup.select("div.grid-col.dmach-grid-item")

data = []

for post in posts:
    # Heading
    heading = post.select_one("h3.entry-title")
    heading = heading.get_text(strip=True) if heading else None

    # Category
    category = post.select_one("p.dmach-postmeta-value a")
    category = category.get_text(strip=True) if category else None

    # State
    state = post.select_one(".dmach-acf-item-content")
    state = state.get_text(strip=True) if state else None

    # Year
    year = post.select_one("p.dmach-acf-value")
    year = year.get_text(strip=True).replace("Year:", "").strip() if year else None

    # Date (if present on page)
    date = post.select_one("span.dmach-post-date")
    date = date.get_text(strip=True) if date else None

    data.append({
        "Heading": heading,
        "State": state,
        "Category": category,
        "Year": year,
        "Date": date
    })

# Save to CSV
df = pd.DataFrame(data)
df.to_csv("output1.csv", index=False)

In [9]:
import pandas as pd
df = pd.read_csv('/content/output1.csv')
display(df)

,Category,Title,State,Year,Date,Description
0,Censorship,"Globally Renowned Scholar of Hindi, Francesca ...",Applicable Across India,2025.0,10/21/2025,NaN
1,Censorship – Academia,"Francesca Orsini, scholar of Hindi and profess...",NaN,NaN,NaN,NaN
2,Arrests,"Rajasthan police detain 2 journalists, Anand P...",Rajasthan,2025.0,10/17/2025,NaN
3,Arrests – Journalists,The Rajasthan Police on Friday picked up two j...,NaN,NaN,NaN,NaN
4,"Censorship, Lawfare","Mumbai Police book, detain TISS students who a...",Maharashtra,2025.0,10/13/2025,NaN
...,...,...,...,...,...,...
448,"Lawfare – National Security, Others","In 2017, environment activist R S Mugilan made...",NaN,NaN,NaN,NaN
449,Lawfare,Summons issued against the editor and deputy e...,Applicable Across India,2016.0,4/26/2016,NaN
450,"Lawfare – Defamation, Journalists","JNU professor, Amita Singh, had filed a compla...",NaN,NaN,NaN,NaN
451,Lawfare,Supreme Court Quashes Criminal Defamation Case...,Karnataka,2014.0,8/22/2014,NaN


In [10]:
df = df.drop(columns=['Description'])
df = df.dropna()
display(df)

,Category,Title,State,Year,Date
0,Censorship,"Globally Renowned Scholar of Hindi, Francesca ...",Applicable Across India,2025.0,10/21/2025
2,Arrests,"Rajasthan police detain 2 journalists, Anand P...",Rajasthan,2025.0,10/17/2025
4,"Censorship, Lawfare","Mumbai Police book, detain TISS students who a...",Maharashtra,2025.0,10/13/2025
6,Lawfare,Independent journalist Ajit Rathi served a leg...,Uttarakhand,2025.0,10/11/2025
8,Killings,Bagalkot journalist Basavaraj Kanakond murdere...,Karnataka,2025.0,10/8/2025
...,...,...,...,...,...
443,Attacks,Bihar: Doctor assaults journalist Shahnawaz Hu...,Bihar,2021.0,8/2/2021
445,Lawfare,Journalist Abhijit Iyer-Mitra files defamation...,Delhi,2019.0,12/14/2019
447,Lawfare,Activist Mugilan booked under sedition charges...,Tamil Nadu,2017.0,4/23/2017
449,Lawfare,Summons issued against the editor and deputy e...,Applicable Across India,2016.0,4/26/2016


In [11]:
df.to_csv("clean_output.csv", index=False)